## Load original

In [1]:
import sys

sys.path.append("../")

In [15]:
out_weights = "../data/agent_st/foundation-model-tl-tt-2x.weights"


In [2]:
import torch


in_model = "../data/VPT-models/2x.model"
in_weights = "../data/VPT-models/foundation-model-2x.weights"
out_weights = "../data/agent_st/foundation-model-tl-tt-2x.weights"

environment = "MineRLBasaltFindCave-v0"
device = "cuda"

old_state_dict = torch.load(in_weights, map_location=device)

old_state_dict

OrderedDict([('net.img_process.cnn.stacks.0.firstconv.layer.weight',
              tensor([[[[ 3.7042e-02, -6.4809e-02, -5.1988e-02],
                        [-8.3571e-02, -1.9411e-02,  4.0863e-02],
                        [-3.2426e-02,  7.7259e-02,  4.8328e-02]],
              
                       [[ 4.0237e-02, -5.8202e-02, -6.2113e-02],
                        [-8.3681e-02, -4.0295e-02,  3.2863e-02],
                        [-2.9362e-02,  6.6627e-02,  7.6046e-02]],
              
                       [[ 3.0695e-02,  2.5573e-02,  5.4368e-03],
                        [ 1.7040e-02, -9.2496e-03, -2.1626e-03],
                        [-2.7389e-03, -2.8656e-02, -1.8573e-02]]],
              
              
                      [[[ 3.1902e-02,  3.0894e-02, -3.6394e-02],
                        [-9.3464e-03, -2.1614e-02, -3.6015e-02],
                        [-3.7105e-02, -2.7153e-02,  1.0896e-02]],
              
                       [[ 7.7584e-02,  1.5076e-02, -1.4173e-02],
      

## Transfer Learning

### Prepare model

In [3]:
from src.lib.policy import MinecraftAgentPolicy as MinecraftAgentPolicy

from src.agent import ACTION_TRANSFORMER_KWARGS
import pickle
from src.lib.actions import ActionTransformer
from gym3.types import DictType, Discrete, TensorType
from src.lib.action_mapping import CameraHierarchicalMapping

agent_parameters = pickle.load(open(in_model, "rb"))
policy_kwargs = agent_parameters["model"]["args"]["net"]["args"]
pi_head_kwargs = agent_parameters["model"]["args"]["pi_head_opts"]
pi_head_kwargs["temperature"] = float(pi_head_kwargs["temperature"])

device = torch.device("cuda")
action_mapper = CameraHierarchicalMapping(n_camera_bins=11)
action_space = action_mapper.get_action_space_update()
action_space = DictType(**action_space)

action_transformer = ActionTransformer(**ACTION_TRANSFORMER_KWARGS)

agent_kwargs = dict(policy_kwargs=policy_kwargs, pi_head_kwargs=pi_head_kwargs, action_space=action_space)

### Prepare weights

In [4]:

# Create a new model with a different architecture
new_model = MinecraftAgentPolicy(**agent_kwargs)

# Initialize a dictionary to keep track of the layers in the new model that will be initialized with weights from the old model
new_state_dict = {}

# Iterate through the state_dict of the old model
for key, value in old_state_dict.items():
    # Check if the layer name in the old model corresponds to a layer in the new model
    if key in new_model.state_dict():
        # If it does, add the layer and its weights to the new_state_dict
        print(f"COPY for {key}")
        new_state_dict[key] = value
    else:
        print(f"DO NOT COPY for {key}")



policy_kwargs={'active_reward_monitors': {'craft_stats': {'args': {'collapse_var': True, 'items': ['planks', 'stick', 'crafting_table', 'wooden_pickaxe', 'stone_pickaxe', 'furnace', 'iron_ingot', 'iron_pickaxe', 'diamond_pickaxe', 'torch']}, 'weight': 0}, 'mine_stats': {'args': {'collapse_var': True, 'items': ['log', 'coal_ore', 'stone', 'iron_ore', 'diamond_ore', 'obsidian']}, 'weight': 0}, 'order_invariant_curriculum': {'args': {'curriculum': {'coal': [5, 0.4], 'cobblestone': [11, 0.09090909090909091], 'crafting_table': [1, 1], 'diamond': [10000, 2.6666666666666665], 'diamond_pickaxe': [10000, 8], 'furnace': [1, 1], 'iron_ingot': [3, 1.3333333333333333], 'iron_ore': [3, 1.3333333333333333], 'iron_pickaxe': [1, 4], 'log': [8, 0.125], 'obsidian': [10000, 16], 'planks': [20, 0.05], 'stick': [16, 0.0625], 'stone_pickaxe': [1, 1], 'torch': [16, 0.125], 'wooden_pickaxe': [1, 1]}}, 'weight': 1}, 'pickup_stats': {'args': {'collapse_var': True, 'items': ['log', 'coal', 'cobblestone', 'iron_or

### Load in

In [5]:
# Update the state_dict of the new model with the layers and weights from the old model
new_model.load_state_dict(new_state_dict, strict=False)



_IncompatibleKeys(missing_keys=['net.add_subtasks.norm.weight', 'net.add_subtasks.norm.bias', 'net.add_subtasks.layers.0.weight', 'net.add_subtasks.layers.1.weight'], unexpected_keys=[])

In [12]:
old_state_dict['net.recurrent_layer.blocks.2.r.orc_block.proj_layer.weight'][10]

tensor([ 0.0188, -0.0292, -0.0034,  ..., -0.0010, -0.0067,  0.0018],
       device='cuda:0')

In [18]:
state_dict = new_model.state_dict()
torch.save(state_dict, out_weights)